In [36]:
import bw2data as bd

bd.projects.set_current("multi-level-tds")

In [37]:
biosphere = bd.Database("biosphere")
biosphere.write(
    {
        ("biosphere", "CO2"): {
            "type": "emission",
            "name": "carbon dioxide",
        },
    }
)

if "foreground" in bd.databases:
    del bd.databases["foreground"] # to make sure we create the foreground from scratch
foreground = bd.Database("foreground")
foreground.register()

if "background_2020" in bd.databases:
    del bd.databases["background_2020"] # to make sure we create the foreground from scratch
background_2020 = bd.Database("background_2020")
background_2020.register()

if "background_2030" in bd.databases:
    del bd.databases["background_2030"] # to make sure we create the foreground from scratch
background_2030 = bd.Database("background_2030")
background_2030.register()

100%|██████████| 1/1 [00:00<00:00, 7244.05it/s]

Vacuuming database 


In [38]:
node_co2 = biosphere.get("CO2")

fu = foreground.new_node("fu", name="fu", unit="unit")
fu['reference product'] = "fu"
fu.save()

first_level_input = foreground.new_node("first_level_input", name="first_level_input", unit="unit")
first_level_input['reference product'] = "first_level_input"
first_level_input.save()

background_process_2020 = background_2020.new_node("background_process", name="background_process", unit="background_process")
background_process_2020['reference product'] = "background_process"
background_process_2020.save()

background_process_2030 = background_2030.new_node("background_process", name="background_process", unit="background_process")
background_process_2030['reference product'] = "background_process"
background_process_2030.save()


In [39]:
fu.new_edge(input=fu, amount=1, type="production").save()
first_level_input_to_fu = fu.new_edge(input=first_level_input, amount=1, type="technosphere")

first_level_input.new_edge(input=first_level_input, amount=1, type="production").save()
background_process_2020_to_first_level_input = first_level_input.new_edge(input=background_process_2020, amount=840, type="technosphere")

background_process_2020.new_edge(input=background_process_2020, amount=1, type="production").save()
background_process_2020.new_edge(input=node_co2, amount=1, type="biosphere").save()

background_process_2030.new_edge(input=background_process_2030, amount=1, type="production").save()
background_process_2030.new_edge(input=node_co2, amount=1, type="biosphere").save()


In [40]:
from bw_temporalis import TemporalDistribution
import numpy as np

td_first_level_input_to_fu = TemporalDistribution(
    date=np.array([-2, -1], dtype="timedelta64[M]"), amount=np.array([0.2, 0.8])
)

td_background_process_2020_to_first_level_input = TemporalDistribution(
    date=np.array([-3, -2], dtype="timedelta64[Y]"), amount=np.array([0.4, 0.6])
)

# td_first_level_input_to_fu = TemporalDistribution(
#     date=np.array([-2, -1], dtype="timedelta64[Y]"), amount=np.array([0.2, 0.8])
# )

# td_background_process_2020_to_first_level_input = TemporalDistribution(
#     date=np.array([-3, -2], dtype="timedelta64[Y]"), amount=np.array([0.4, 0.6])
# )

In [41]:
first_level_input_to_fu["temporal_distribution"] = td_first_level_input_to_fu
first_level_input_to_fu.save()

background_process_2020_to_first_level_input["temporal_distribution"] = td_background_process_2020_to_first_level_input
background_process_2020_to_first_level_input.save()

## LCA using `bw_timex`


As usual, we need to select a method:

In [42]:
method = ("GWP", "example")
bd.Method(method).write(
    [
        (("biosphere", "CO2"), 1),
    ]
)

In [43]:
from datetime import datetime

database_date_dict = {
    "background_2020": datetime.strptime("2020", "%Y"),
    "background_2030": datetime.strptime("2030", "%Y"),
    "foreground": "dynamic", # flag databases that should be temporally distributed with "dynamic"
}

In [44]:
from bw_timex import TimexLCA

tlca = TimexLCA({fu: 1}, method, database_date_dict)

In [45]:
1008/3

336.0

In [46]:
1008-840

168

In [50]:
tlca.build_timeline(temporal_grouping="year")

Starting graph traversal
Calculation count: 2


/Users/timodiepers/Documents/Coding/bw_timex/bw_timex/timex_lca.py:194: UserWarning: No edge filter function provided. Skipping all edges within background databases.
  warnings.warn(


,date_producer,producer_name,date_consumer,consumer_name,amount,interpolation_weights
0,2021-01-01,background_process,2024-01-01,first_level_input,672.0,"{'background_2020': 0.8998083766767041, 'backg..."
1,2022-01-01,background_process,2024-01-01,first_level_input,1008.0,"{'background_2020': 0.7998905009581166, 'backg..."
2,2024-01-01,fu,2024-01-01,-1,1.0,None
3,2024-01-01,first_level_input,2024-01-01,fu,1.0,None


In [33]:
tlca.lci()

/Users/timodiepers/anaconda3/envs/timex/lib/python3.10/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)


In [75]:
tlca.lci()

/Users/timodiepers/anaconda3/envs/timex/lib/python3.10/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)


In [78]:
tlca.static_lcia()
tlca.static_score

1680.0000000000002

In [79]:
tlca.static_lca.score

840.0000000000001

In [76]:
tlca.activity_time_mapping_dict

{(('foreground', 'fu'), 'dynamic'): 187,
 (('foreground', 'first_level_input'), 'dynamic'): 188,
 (('background_2020', 'background_process'), 2020): 189,
 (('background_2030', 'background_process'), 2030): 190,
 (('temporalized', 'background_process'), 2021): 191,
 (('temporalized', 'background_process'), 2022): 192,
 (('temporalized', 'fu'), 2024): 193,
 (('temporalized', 'first_level_input'), 2024): 194}

In [77]:
import pandas as pd

expected_inventory = pd.DataFrame(
    data={
        "date": pd.Series(
            data=[
                "01-01-2019",
                "01-01-2020",
                "01-01-2021",
            ],
            dtype="datetime64[s]",
        ),
        "amount": pd.Series(
            data=[0.2*336, 0.8*336+0.2*504, 0.8*504], dtype="float64"
        ),
        "flow": pd.Series(data=[41, 41, 41], dtype="int"),
        "activity": pd.Series(data=[46, 44, 47], dtype="int"),
    }
)
expected_inventory

,date,amount,flow,activity
0,2019-01-01,67.2,41,46
1,2020-01-01,369.6,41,44
2,2021-01-01,403.2,41,47


In [49]:
tlca.create_labelled_technosphere_dataframe()

,"((foreground, fu), dynamic)","((foreground, first_level_input), dynamic)","((background_2020, background_process), 2020)","((background_2030, background_process), 2030)","((temporalized, background_process), 2019)","((temporalized, background_process), 2020)","((temporalized, background_process), 2021)","((temporalized, first_level_input), 2022)","((temporalized, first_level_input), 2023)","((temporalized, fu), 2024)"
"((foreground, fu), dynamic)",1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
"((foreground, first_level_input), dynamic)",-1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
"((background_2020, background_process), 2020)",0.0,-840.0,1.0,0.0,-1.0,-1.0,-0.899808,0.0,0.0,0.0
"((background_2030, background_process), 2030)",0.0,0.0,0.0,1.0,0.0,0.0,-0.100192,0.0,0.0,0.0
"((temporalized, background_process), 2019)",0.0,0.0,0.0,0.0,1.0,0.0,0.000000,-336.0,0.0,0.0
"((temporalized, background_process), 2020)",0.0,0.0,0.0,0.0,0.0,1.0,0.000000,-504.0,-336.0,0.0
"((temporalized, background_process), 2021)",0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,-504.0,0.0
"((temporalized, first_level_input), 2022)",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,-0.2
"((temporalized, first_level_input), 2023)",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,-0.8
"((temporalized, fu), 2024)",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0


In [50]:
tlca.create_labelled_dynamic_biosphere_dataframe()

,"((foreground, fu), dynamic)","((foreground, first_level_input), dynamic)","((background_2020, background_process), 2020)","((background_2030, background_process), 2030)","((temporalized, background_process), 2019)","((temporalized, background_process), 2020)","((temporalized, background_process), 2021)","((temporalized, first_level_input), 2022)","((temporalized, first_level_input), 2023)","((temporalized, fu), 2024)"
"(181, 2019-01-01T00:00:00)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"(181, 2020-01-01T00:00:00)",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(181, 2021-01-01T00:00:00)",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [51]:
from dynamic_characterization.timex import characterize_co2

In [53]:
tlca.dynamic_lcia(metric="GWP", characterization_function_dict={node_co2.id: characterize_co2})

/Users/timodiepers/anaconda3/envs/timex/lib/python3.10/site-packages/dynamic_characterization/dynamic_characterization.py:262: UserWarning: Using bw_timex's default CO2 characterization function for GWP reference.
  warnings.warn(


,date,amount,flow,activity
0,2019-01-01,67.2,181,186
1,2020-01-01,369.6,181,187
2,2021-01-01,403.2,181,188


In [54]:
tlca.dynamic_score

840.0

In [55]:
tlca.static_lca.score

840.0000000000001

In [56]:
tlca.static_lcia()
tlca.static_score

840.0